In [2]:
import pandas as pd

offical_trainX = pd.read_csv("./data/dengue_features_train.csv")
offical_trainY = pd.read_csv("./data/dengue_labels_train.csv")

trimmed_trainX=offical_trainX.drop(['week_start_date'], axis=1)
trimmed_trainY=offical_trainY.drop(['year', 'weekofyear'], axis=1)

sj_official_trainX= trimmed_trainX[trimmed_trainX.city == "sj"]
iq_official_trainX= trimmed_trainX[trimmed_trainX.city == "iq"]

sj_len_X=len(sj_official_trainX)
iq_len_X=len(iq_official_trainX)

sj_trainX=sj_official_trainX[:749]
iq_trainX=iq_official_trainX[:416]

sj_testX=sj_official_trainX[749:]
iq_testX=iq_official_trainX[416:]

sj_official_trainY= trimmed_trainY[trimmed_trainY.city == "sj"]
iq_official_trainY= trimmed_trainY[trimmed_trainY.city == "iq"]

sj_len_Y=len(sj_official_trainY)
iq_len_Y=len(sj_official_trainY)

sj_trainY=sj_official_trainY[:749]
iq_trainY=iq_official_trainY[:416]

sj_testY=sj_official_trainY[749:]
iq_testY=iq_official_trainY[416:]

trainX = pd.concat((sj_trainX, iq_trainX), axis=0)
trainY = pd.concat((sj_trainY, iq_trainY), axis=0)
testX = pd.concat((sj_testX, iq_testX), axis=0)
testY = pd.concat((sj_testY, iq_testY), axis=0)

trainY=trainY.drop('city', axis=1)
testY=testY.drop('city', axis=1)


In [ ]:
trainX.info()

In [ ]:
trainX.describe().T

In [ ]:
trainX.isnull().sum()

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings("ignore")

all_numerical_features = trainX.select_dtypes(include=["int64", "float64"]).columns
all_categorical_features = trainX.select_dtypes(include=[object]).columns
numerical_features = [value for value in all_numerical_features]
categorical_features = [value for value in all_categorical_features]

total_abs_error = 0
runtime_amount = 30

def best_pipeline_intown(trainX, trainY, testX): ### returns predicted Y

    # Preprocessing for numerical data
    numerical_transformer = Pipeline(
        steps=[("imputer", KNNImputer(n_neighbors=5)), 
               ("scaler", StandardScaler())]
    )
    # Preprocessing for categorical data
    categorical_transformer = OneHotEncoder()

    # Bundle Preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, numerical_features),
            ("cat", OneHotEncoder(), categorical_features),
        ],
        remainder="passthrough",
    )

    model = RandomForestRegressor(n_estimators=300, max_depth=5)
    # model = SVR()

    pipeline = Pipeline([("preprocessor", preprocessor), ("regressor", model)])
    pipeline.fit(trainX, trainY)
    predicted_Y = pipeline.predict(testX)
    
    return predicted_Y

for i in range(runtime_amount):
    predicted_Y = best_pipeline_intown(trainX, trainY, testX)
    
    absolute_error = mean_absolute_error(testY, predicted_Y)
    total_abs_error += absolute_error
    print(i + 1, "MAE:", absolute_error)

print("Average MAE:", total_abs_error / runtime_amount)


1 MAE: 13.178828975619712
2 MAE: 13.147897963953382
3 MAE: 13.09677516254512
4 MAE: 13.115003930183413
5 MAE: 13.130026498938092
6 MAE: 13.12745365564165
7 MAE: 13.119409384982749
8 MAE: 13.089627445324162
9 MAE: 13.14796100507293


KeyboardInterrupt: 